In [3]:
import ast
from pathlib import Path

from mutation import maker

In [4]:
pkg_dir = Path("firstmodule")

In [6]:
src_files = maker.get_py_files(pkg_dir)
print(*src_files, sep="\n")

/home/zassoc/Github/m/firstmodule/__init__.py
/home/zassoc/Github/m/firstmodule/test_run.py
/home/zassoc/Github/m/firstmodule/run.py


In [7]:
tree = maker.get_ast_tree(src_files[-1])

In [8]:
tree.body

In [42]:
class RewriteAddSub(ast.NodeTransformer):
    
    def __init__(self, readonly):
        self.readonly = readonly
        self.locs = []
        super()
        
    def visit_BinOp(self, node):
        self.generic_visit(node)
        
        self.locs.append(node.lineno)
        if self.readonly:
            print("Read only", node.lineno, node.col_offset, node.op)    
            self.locs.append(node.op)
            return node
        
        else:
            print("Transform to sub")
            return ast.copy_location(
                ast.BinOp(left=node.left,
                          op=ast.Sub(),
                          right=node.right),
                node)

In [43]:
ronly = RewriteAddSub(readonly=True)

In [44]:
t2 = ronly.visit(tree)

Read only 6 11 <_ast.Add object at 0x7f797947e860>
Read only 10 11 <_ast.Add object at 0x7f797947e860>


In [45]:
ronly.locs

[6, <_ast.Add at 0x7f797947e860>, 10, <_ast.Add at 0x7f797947e860>]

In [57]:
type(ronly.locs[1])

_ast.Add

In [59]:
isinstance(ronly.locs[1], type(ronly.locs[1]))

True

In [63]:
repr(type(ronly.locs[1]))

SyntaxError: invalid syntax (<string>, line 1)